In [2]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
#import seaborn as sns
from pylab import rcParams
import re
import unicodedata  # remove acentos
import nltk
import spacy
import csv
import sys
#import torch as cu
import torch
from bertopic import BERTopic
#from cuml.cluster import HDBSCAN
#from cuml.manifold import UMAP
#from numba import vectorize
#from inspect import signature

In [ ]:
def Salvar(result, nome_arq):
    arq = 'Base_acordaos/Resultados/'+nome_arq
    cabecalho = ('numero_processo', 'id_documento', 'Tema_Real', 'Tema_Classificado')
    with open(arq, 'w', newline = '') as csvfile:
        my_writer = csv.writer(csvfile, delimiter = ',')
        my_writer.writerow(cabecalho)
        for aux in result:
            my_writer = csv.writer(csvfile, delimiter = ',')
            my_writer.writerow(aux)
    print('Done')


In [ ]:
def Codigo_para_Tema(codigo):
    if('85763' in codigo):
        return '1076'
    elif('85721' in codigo):
        return '1039'
    elif('85738' in codigo):
        return '1101'
    elif('85728' in codigo):
        return '1046'
    elif('85714' in codigo):
        return '1033'
    elif('85696' in codigo):
        return '1015'
    elif('85568'in codigo):
        return '929'

In [ ]:
#base_test é o conjunto de acórdãos que será trabalhado como teste (3k ou qualquer fold de teste)
def Classification_result(base_test, result, file_name):
    classification = []
    for i in range(len(result)):
        id_process = result[i][0]
        print('id_process =', id_process)
        id_doc = result[i][1]
        print('id_doc =', id_doc)
        tema_real = Codigo_para_Tema(str(base_test.loc[i].at['codigos_movimentos_temas']))
        print('tema_real = ', tema_real)
        tema_classificado = result[i][2]
        print('tema_classificado = ', tema_classificado)
        classification.append((id_process, id_doc, tema_real, tema_classificado))
        print('tamanho =', len(classification))
        
    Salvar(classification, file_name)

In [ ]:
#!python -m spacy download pt_core_news_lg

In [ ]:
nlp = spacy.load('pt_core_news_lg')


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Usando",device, "device")

Usando cuda device


**Instalação do Sentence-Transformer**

In [ ]:
#%pip install -U sentence-transformers


**Pré-Processamento**

In [ ]:
class Preprocessing():

    def __init__(self, df, name_column, threshold=10, remove_stopwords=False, stopwords_language="portuguese",
                 ugc=False, stemming=False, remove_emojis=False, identation=False, html=False):
        '''
        INPUTS
        :param df (pandas dataframe): the pandas dataframe that is going to be preprocessed
        :param name_column (string): the column of the dataframe that is going to be preprocessed
        :param threshold (integer): if the string is smaller than the threshold, the instance is removed (default: 10)
        :param remove_stopwords (boolean): if the stop words are going to be removed (True), otherwise (False) (default: False)
        :param ugc (boolean): if there is user-generated content (True - to remove hashtags, mentions), otherwise (False) (default: True)
        :param stemming (boolean): if the words are going to be stemmed (True), otherwise (False) (default: False)
        :param remove_emojis (boolean): if the emojis are going to be removed (True), otherwise (False) (default: True)
        :param indentation (boolean): if the text has indentation symbols (e.g., \t, \n) it is true, otherwise (False) (default: False)
        :param html (boolean): if the text has HTML symbols (e.g., <a, <br>) it is true, otherwise (False) (default: False)

        OUTPUT
        :param df_final (pandas dataframe): the preprocessed pandas dataframe
        '''
        self.df = df
        self.name_column = name_column
        self.threshold = threshold
        self.remove_stopwords = remove_stopwords
        self.language_stopwords = stopwords_language
        self.ugc = ugc
        self.stemming = stemming
        self.remove_emojis = remove_emojis
        self.identation = identation
        self.html = html

    def preprocess(self):

        !pip3 install - -user - U nltk
        import nltk
        import math
        import re

        if (self.remove_stopwords == True):
            from nltk.corpus import stopwords
            nltk.download('stopwords')
            list_stopwords = stopwords.words(self.language_stopwords)
            sw = [word for word in list_stopwords if (
                word != 'no' and word != 'nor' and word != 'not')]

        if (self.stemming == True):
            from nltk.stem.snowball import SnowballStemmer
            stemmer = SnowballStemmer(self.language_stopwords)

        if (self.ugc == True and self.remove_emojis == True):  # https://pypi.org/project/demoji/
            import tarfile
            # downloaded at https://pypi.org/project/demoji/#files
            !pip3 install demoji-0.3.0-py2.py3-none-any.whl
            # downloaded at https://pypi.org/project/demoji/#files
            tar = tarfile.open("demoji-0.3.0.tar.gz", "r:gz")
            for member in tar.getmembers():
                f = tar.extractfile(member)
                if f is not None:
                    content = f.read()
            !python - m pip install demoji
            import demoji
            demoji.download_codes()

        df_dataframe = self.df
        nan_items = []

        for item in range(len(df_dataframe)):
            try:
                if math.isnan(df_dataframe.loc[item, self.name_column]) == True:
                    nan_items.append(item)
            except:
                pass

        df_dataframe = df_dataframe.drop(
            axis=0, index=nan_items)  # it removes empty cells
        df_dataframe = df_dataframe.drop_duplicates(
            subset=[self.name_column])  # it removes duplicate cells
        size_df_dataframe = int(len(df_dataframe))
        # it rearranges the indices of the dataframe
        df_dataframe = df_dataframe.set_index(
            pd.Series(range(0, size_df_dataframe)))
        list_index = [item for item in range(size_df_dataframe) if (int(len(
            df_dataframe.at[df_dataframe.index[item], self.name_column])) <= self.threshold)]
        # it removes the remaining instances smaller than the threshold
        df_dataframe = df_dataframe.drop(axis=0, index=list_index)
        size_df_dataframe = int(len(df_dataframe))
        # it rearranges the indices of the dataframe
        df_dataframe = df_dataframe.set_index(
            pd.Series(range(0, size_df_dataframe)))
        df_heads = list(df_dataframe.columns)  # it saves the header
        # it storages the position of the column that was chosen based on the input (string)
        position = df_heads.index(self.name_column)
        numpy_array = np.array(df_dataframe)
        size = int(len(numpy_array))

        for i in range(size):
            tokens = []
            string = numpy_array[i][position]
            # it removes links
            string = re.sub(
                '(https?:(\/\/))?(www\.)?([a-zA-Z0-9]+[/\.:#%~&?=\_\-,]+){2,}[a-zA-Z0-9]*', '', string)

            if (self.identation == True):  # it removes some identation marks
                # it removes \b \f \n \r \t
                string = re.sub(r'(\\b|\\f|\\n|\\r|\\t)', '', string)
                string = re.sub(r'(\x08)', '', string)

            if (self.html == True):  # it only removes some basic tags
                # depending on the variety of tags - use HTML parsers such as BeautifulSoup https://www.crummy.com/software/BeautifulSoup/bs4/doc/
                string = re.sub(
                    r'((\<(\/)?a(\>)?)|(\<(\/)?body(\>)?)|(\<br\>)|(class(\s)?\=)|(\<(\/)?div(\>)?)|(\<(\/)?form(\>)?)|(href(\s)?\=)|(id(\s)?\=)|(\<(\/)?link(\>)?)|(\<(\/)?p(\s)?(\/)?(\>)))', '', string)
                string = re.sub(r'(\x08)', '', string)

            # it removes some special characters - $¹²³£¢¬%¨&*()=+§ªº°/\|~^{}[]_><
            string = re.sub(
                r'(\$|\¹|\²|\³|\£|\¢|\¬|\%|\¨|\&|\*|\(|\)|\=|\+|\§|\ª|\º|\°|\/|\\|\||\~|\^|\{|\}|\[|\]|\_|\>|\<)', '', string)
            # they remove repeated punctuation
            # it removes repeated question marks
            string = re.sub(r'((\?)+)', '?', string)
            # it removes repeated interrobangs
            string = re.sub(r'((\?\!)+)', '?!', string)
            # it removes repeated interrobangs
            string = re.sub(r'((\!\?)+)', '?!', string)
            # it removes repeated exclamation marks
            string = re.sub(r'((\!)+)', '!', string)
            # it removes repeated punctuation bigger than ellipsis
            string = re.sub(r'((\.\.\.\.)+)', '...', string)
            tokens = string.split()  # it splits the instances on a list of tokens
            tokens_range = int(len(tokens))
            tokens = [tokens[item].lower()
                      for item in range(tokens_range)]  # case-folding

            if (self.ugc == True):  # if the instances are user-generated content
                tokens = [('') if ((len(tokens[item]) > 1) and (tokens[item][0] == '@' or tokens[item]
                                                                [1] == '@')) else tokens[item] for item in range(tokens_range)]  # it removes mentions
                tokens = [('at') if (len(tokens[item]) == 1 and tokens[item][0] == '@')
                          else tokens[item] for item in range(tokens_range)]  # it replaces @ with 'at'
                tokens = [('to') if (len(tokens[item]) == 1 and tokens[item][0] == '2')
                          else tokens[item] for item in range(tokens_range)]  # it replaces 2 with 'to'
                tokens = [('for' + tokens[item][1:]) if (len(tokens[item]) >= 1 and tokens[item][0] == '4')
                          else tokens[item] for item in range(tokens_range)]  # it replaces 4 with 'for'
                tokens = [('you') if (len(tokens[item]) == 1 and tokens[item][0] == 'u') else tokens[item]
                          for item in range(tokens_range)]  # it replaces strings with one character 'u' with 'you'
                tokens = [(tokens[item][1:]) if (tokens[item] != '' and tokens[item][0] == '#' and len(tokens[item]) > 0) else (
                    tokens[item]) for item in range(tokens_range)]  # it removes the '#' but keeps the remaining string
                tokens = [re.sub(r'((h(a)+)+(h(a)+)+)|((h(e)+)+(h(e)+)+)|((h(i)+)+(h(i)+)+)',
                                 '', tokens[item]) for item in range(tokens_range)]  # it removes laughter
                tokens = [re.sub(r'(((aaa)+|(eee)+|(ii)+|(ooo)+|(uu)+|(yy)+))', '', tokens[item])
                          for item in range(tokens_range)]  # it removes some repeated vowels and semivowels

                if (self.remove_emojis == True):  # it removes emojis - https://pypi.org/project/demoji/
                    for item in range(tokens_range):
                        if tokens[item] != '':
                            a = demoji.findall(tokens[item])
                            size = len(a)
                            if size > 0:  # if there are emojis
                                for key in a.keys():
                                    tokens[item] = re.sub(
                                        key, '', tokens[item])

            if (self.remove_stopwords == True):  # it removes stop words
                tokens = [('') if (tokens[item] in sw) else (tokens[item])
                          for item in range(tokens_range)]

            if (self.stemming == True):  # it stems the words
                tokens = [stemmer.stem(tokens[item]) if (
                    tokens[item] != "") else ("") for item in range(tokens_range)]

            aux = " ".join(tokens).split()
            numpy_array[i][position] = " ".join(aux)

        df_final = pd.DataFrame(numpy_array, columns=df_heads)
        nan_items = []

        for item in range(len(df_final)):
            try:
                if math.isnan(df_final.loc[item, self.name_column]) == True:
                    nan_items.append(item)
            except:
                pass

        # it removes empty cells
        df_final = df_final.drop(axis=0, index=nan_items)
        list_index = [item for item in range(len(df_final)) if (
            len(df_final.at[df_final.index[item], self.name_column]) <= self.threshold)]
        # it removes the remaining instances smaller than the threshold
        df_final = df_final.drop(axis=0, index=list_index)
        # it rearranges the indices of the dataframe
        df_final = df_final.set_index(pd.Series(range(0, len(df_final))))
        df_final = df_final.drop_duplicates(
            subset=[self.name_column])  # it removes duplicate cells
        # it rearranges the indices of the dataframe
        df_final = df_final.set_index(pd.Series(range(0, len(df_final))))
        list_index = [item for item in range(len(df_final)) if (
            int(len(df_final.at[df_final.index[item], self.name_column])) <= self.threshold)]
        # it removes the remaining instances smaller than the threshold
        df_final = df_final.drop(axis=0, index=list_index)
        # it rearranges the indices of the dataframe
        df_final = df_final.set_index(pd.Series(range(0,len(df_final)))) # it rearranges the indices of the dataframe

        return df_final

**Carregando Acórdãos**

In [ ]:
#%pip install -U pyarrow

In [ ]:
#######CARREGANDO ARQUIVOS .PARQUET###########

#path ='/content/drive/My Drive/Colab Notebooks/acordaos_tema_exclusivo/Base_v0/'
'''
dff=pd.read_parquet(path + 'dff.parquet.gzip', engine='pyarrow')
'''
# OU
'''
file = '/content/drive/My Drive/Colab Notebooks/base_acordaos_2_pre_proc.parquet' 
dff = pd.read_parquet(file)
'''
'''
file = 'Base_acordaos/df_test_with_segments.parquet.gzip'
dff=pd.read_parquet(file, engine='pyarrow')
'''

######CARREGANDO ARQUIVOS .TXT################

#with open (path + 'acordaos_clean.txt', 'r') as arquivo:
#  arq_txt = arquivo.readlines()

'''
with open(path + 'acordaos_clean.txt') as arquivo:
  arq_txt = arquivo.readlines()
arquivo.close()
  
type(arq_txt)
'''

######CARREGANDO ARQUIVOS .CSV################

#arq = 'Base_acordaos/acordaos_felipe_ultima.csv'
#arq = '/home/marcio/Downloads/tabela-ultima.csv'
#arq = 'Base_acordaos/acordaos_principais_14k_formatado.csv'
#arq = 'Base_acordaos/acordaos_principais_3k_formatado.csv'
#arq = 'Base_acordaos/acordaos_principais_40k_formatado.csv'
#arq = 'Base_acordaos/acordaos_principais_fila_formatado.csv'

arq = 'Base_acordaos/acordaos_principais_3k_3_C_limpo.csv'

dff = pd.read_csv(arq, sep=",")



#caminhos = [os.path.join('/content/drive/My Drive/Colab Notebooks/acordaos_tema_exclusivo/1101/', nome) for nome in os.listdir('/content/drive/My Drive/Colab Notebooks/acordaos_tema_exclusivo/1101/')]
#arquivos = [arq for arq in caminhos if os.path.isfile(arq)]
#len(arquivos)
dff.head()


,numero_processo,id_documento,data_hora_documento,natureza_processo,codigos_movimentos_temas,conteudo,formatado
0,00001508420158260315,76846788 - 0,2020-02-10 00:00:00,F,85738,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,"Registro: 2020.0000079227 ACÓRDÃO Vistos, rel..."
1,00003494520158260691,55161519 - 0,2018-07-31 00:00:00,D,85721,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,DO ESTADO DE SÃO PAULO Registro: 2018.0000563...
2,00004868820058260590,12628095 - 0,2014-02-21 00:00:00,F,85721,TRIBUNAL DE JUSTIÇA\n PODER JUDICIÁRIO\n São ...,"Registro: 2014.0000095962 ACÓRDÃO Vistos, rel..."
3,00005505720218260196,114316205 - 0,2022-02-21 00:00:00,D,85721,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,"Registro: 2022.0000110428 ACÓRDÃO Vistos, rel..."
4,00005702020168260262,55185431 - 0,2018-07-31 00:00:00,F,85721,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,DO ESTADO DE SÃO PAULO Registro: 2018.0000563...


CÓDIGO DOS TEMAS

código_movimento      código_na_fonte
85763                 S1076
85721                 S1039
85738                 S1101
85728                 S1046
85714                 S1033
85696                 S1015
85568                 S0929


In [ ]:
aux = str(dff.iloc[0].loc['codigos_movimentos_temas'])
aux

'85738'

**VERIFICANDO QUANTOS DOCS SÃO MULTILABEL**

In [ ]:
'''
cont = 0
for i in range(len(dff)):
    if (len(dff.iloc[i].at['codigos_movimentos_temas']) > 5):
        cont = cont + 1
print('Quantidade de documentos com multilabel é',cont)
'''

"\ncont = 0\nfor i in range(len(dff)):\n    if (len(dff.iloc[i].at['codigos_movimentos_temas']) > 5):\n        cont = cont + 1\nprint('Quantidade de documentos com multilabel é',cont)\n"

**QUANT. DE DOCS COM APENAS UM LABEL POR DOC**

In [ ]:
#len(dff.iloc[4].at['codigos_movimentos_temas'])
s1076 = 0
s1039 = 0
s1101 = 0
s1046 = 0
s1033 = 0
s1015 = 0
s0929 = 0


for i in range(len(dff)):
    if(str(dff.iloc[i].at['codigos_movimentos_temas']) == '85763'):
        s1076 = s1076 + 1
    elif(str(dff.iloc[i].at['codigos_movimentos_temas']) == '85721'):
        s1039 = s1039 + 1
    elif(str(dff.iloc[i].at['codigos_movimentos_temas']) == '85738'):
        s1101 = s1101 + 1
    elif(str(dff.iloc[i].at['codigos_movimentos_temas']) == '85728'):
        s1046 = s1046 + 1
    elif(str(dff.iloc[i].at['codigos_movimentos_temas']) == '85714'):
        s1033 = s1033 + 1
    elif(str(dff.iloc[i].at['codigos_movimentos_temas']) == '85696'):
        s1015 = s1015 + 1
    elif(str(dff.iloc[i].at['codigos_movimentos_temas']) == '85568'):
        s0929 = s0929 + 1

In [ ]:
print('Quantidade de documentos com apenas o tema 1076 é',s1076)
print('Quantidade de documentos com apenas o tema 1039 é',s1039)
print('Quantidade de documentos com apenas o tema 1101 é',s1101)
print('Quantidade de documentos com apenas o tema 1046 é',s1046)
print('Quantidade de documentos com apenas o tema 1033 é',s1033)
print('Quantidade de documentos com apenas o tema 1015 é',s1015)
print('Quantidade de documentos com apenas o tema 929 é',s0929)

Quantidade de documentos com apenas o tema 1076 é 0
Quantidade de documentos com apenas o tema 1039 é 128
Quantidade de documentos com apenas o tema 1101 é 534
Quantidade de documentos com apenas o tema 1046 é 0
Quantidade de documentos com apenas o tema 1033 é 90
Quantidade de documentos com apenas o tema 1015 é 6
Quantidade de documentos com apenas o tema 929 é 579


In [ ]:
len(dff)

1337

In [ ]:
print(dff.iloc[0].at['formatado'])

 Registro: 2020.0000079227 ACÓRDÃO Vistos, relatados e discutidos estes autos de Apelação Cível nº 0000150-84.2015.8.26.0315, da Comarca de Laranjal Paulista, em que é apelante BANCO DO BRASIL S/A, é apelado ANA CRISTIANE CARNIEL BRONZATTI. ACORDAM, em sessão permanente e virtual da 17ª Câmara de Direito Privado do Tribunal de Justiça de São Paulo, proferir a seguinte decisão: Deram provimento em parte ao recurso. V. U., de conformidade com o voto do relator, que integra este acórdão. O julgamento teve a participação dos Desembargadores IRINEU FAVA (Presidente) e SOUZA LOPES. São Paulo, 10 de fevereiro de 2020. JOÃO BATISTA VILHENA Relator Assinatura Eletrônica APELANTE: BANCO DO BRASIL S/A APELADO: ANA CRISTIANE CARNIEL BRONZATTI VOTO nº97.406 EMENTA APELAÇÃO EXPURGOS INFLACIONÁRIOS AÇÃO CIVIL PÚBLICA EXECUÇÃO INDIVIDUAL Fase de liquidação de sentença Necessidade de observância do disposto no art. 509, inc. II, do CPC de 2015 Caso concreto em que efetivamente não houve qualquer prejuí

**Extração de Sentença**

In [ ]:
#version 1 - NLTK

def Segmentation_v1(txt): 
  
  from nltk.tokenize import sent_tokenize

  segs = sent_tokenize(txt)

  return segs

In [ ]:
#version 2 - SpacyProjetor

def Segmentation_v2(txt): 
  sentencas = []
  doc = nlp(txt)
  for sent in doc.sents:
    sentencas.append(str(sent))
  return sentencas

**Funções de Descrição dos Temas**

In [ ]:
#Versão Completa da descrição dos Temas (segundo o STJ) - Sem o uso do modelo NLP

def description_full(): 
  desc_tema1101 = 'termo final da incidência dos juros remuneratórios nos casos de ações coletivas e individuais reivindicando a reposição de expurgos inflacionários em cadernetas de poupança.'
  desc_tema1039 = 'fixação do termo inicial da prescrição da pretensão indenizatória em face de seguradora nos contratos, ativos ou extintos, do sistema financeiro de habitação.'
  desc_tema1033 = 'interrupção do prazo prescricional para pleitear o cumprimento de sentença coletiva, em virtude do ajuizamento de ação de protesto ou de execução coletiva por legitimado para propor demandas coletivas.'
  desc_tema1015 = 'legitimidade passiva do hsbc bank brasil s/a para responder pelos encargos advindos de expurgos inflacionários relativos a cadernetas de poupança mantidas perante o extinto banco bamerindus s/a, em decorrência de sucessão empresarial havida entre as instituições financeiras.'
  desc_tema929 = 'discussão quanto às hipóteses de aplicação da repetição em dobro prevista no art. 42, parágrafo único, do CDC.'
  #desc_tema1046 = 'a possibilidade de fixação de honorários advocatícios com fundamento em juízo de equidade, nos termos do art. 85, §§ 2º e 8º, do código de processo civil de 2015.'
  
  desc_temas = [desc_tema1101, desc_tema1039, desc_tema1033, desc_tema1015, desc_tema929]#, desc_tema1046]
  return desc_temas

In [ ]:
#Versão Completa da descrição dos Temas (segundo o STJ) - Com o uso do modelo spacy NLP, para fazer o cálculo da similaridade utilizando o spacy 

def description_full_nlp(): #Com o uso do pacote NLP
  desc_tema1101 = nlp(u'termo final da incidência dos juros remuneratórios nos casos de ações coletivas e individuais reivindicando a reposição de expurgos inflacionários em cadernetas de poupança.')
  desc_tema1039 = nlp(u'fixação do termo inicial da prescrição da pretensão indenizatória em face de seguradora nos contratos, ativos ou extintos, do sistema financeiro de habitação.')
  desc_tema1033 = nlp(u'interrupção do prazo prescricional para pleitear o cumprimento de sentença coletiva, em virtude do ajuizamento de ação de protesto ou de execução coletiva por legitimado para propor demandas coletivas.')
  desc_tema1015 = nlp(u'legitimidade passiva do hsbc bank brasil s/a para responder pelos encargos advindos de expurgos inflacionários relativos a cadernetas de poupança mantidas perante o extinto banco bamerindus s/a, em decorrência de sucessão empresarial havida entre as instituições financeiras.')
  desc_tema929 = nlp(u'discussão quanto às hipóteses de aplicação da repetição em dobro prevista no art. 42, parágrafo único, do CDC.')
  #desc_tema1046 = nlp(u'a possibilidade de fixação de honorários advocatícios com fundamento em juízo de equidade, nos termos do art. 85, §§ 2º e 8º, do código de processo civil de 2015.')
 
  desc_temas = [desc_tema1101, desc_tema1039, desc_tema1033, desc_tema1015, desc_tema929]#, desc_tema1046]
  return desc_temas

In [ ]:
#Versão Resumida (segundo o TJ) - sem stopwords e sem o uso do modelo spacy NLP

def description_summarized(): 
  desc_tema1101_res = 'expurgos inflacionários juros remuneratórios termo final'
  desc_tema1039_res = 'sfh termo prescrição indenização'
  desc_tema1033_res = 'acp liquidação prescrição interrupção protesto bbdf'
  desc_tema1015_res = 'acp hsbc legitimidade sucessão bamerindus s/a'
  desc_tema929_res = 'repetição dobro pagamento art. 42 cdc'
  #desc_tema1046_res = 'honorários advocatícios equidade - artigo 85 do CPC' 

  desc_temas_res = [desc_tema1101_res, desc_tema1039_res, desc_tema1033_res, desc_tema1015_res, desc_tema929_res]#, desc_tema1046_res]
  return desc_temas_res

In [ ]:
#Versão Resumida (segundo o TJ) - sem stopwords e com o uso do modelo spacy NLP

def description_summarized_nlp(): 
  desc_tema1101_res = nlp(u'expurgos inflacionários juros remuneratórios termo final')
  desc_tema1039_res = nlp(u'sfh termo prescrição indenização')
  desc_tema1033_res = nlp(u'acp liquidação prescrição interrupção protesto bbdf')
  desc_tema1015_res = nlp(u'acp hsbc legitimidade sucessão bamerindus s/a')
  desc_tema929_res = nlp(u'repetição dobro pagamento art. 42 cdc')
  #desc_tema1046_res = nlp(u'honorários advocatícios equidade - artigo 85 do CPC') 

  desc_temas_res = [desc_tema1101_res, desc_tema1039_res, desc_tema1033_res, desc_tema1015_res, desc_tema929_res]#, desc_tema1046_res]
  return desc_temas_res

**Funções de similaridade - Calculando Média das Similaridade dos termos do tópico mais frequente com a descrição de cada Tema**

In [ ]:
#Similaridade Média usando Spacy

def similaridade_spacy_media(top):  
  
  desc_temas = description_full()
  #desc_temas_res = description_summarized()
  media = []
  for y in range(len(desc_temas)):
  #for y in range(len(desc_temas_res)):
    sim = 0.0
    for x in range(len(top)):
      tp = nlp(top[x])
      tm = nlp(desc_temas[y])
      #tm = nlp(desc_temas_res[y])
      sim = sim + tp.similarity(tm)
    media.append(sim/10.0)
  return media

In [ ]:
#Similaridade Média usando Sentence Transformer v1 (pytorch)

def similaridade_Transformer_media_v1(topicos):
  from sentence_transformers import SentenceTransformer,util
  model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
  

  desc_temas = description_full()
  #desc_temas_res = description_summarized()
  
  topics_embeddings = model.encode(topicos)
  desc_temas_embeddings = model.encode(desc_temas)
  #desc_temas_res_embeddings = model.encode(desc_temas_res)

  media = []
  #for y in range(len(desc_temas_res_embeddings)):
  for y in range(len(desc_temas_embeddings)):
    sim = 0.0
    for x in range(len(topics_embeddings)):
      sim = sim + util.pytorch_cos_sim(desc_temas_embeddings[y], topics_embeddings[x])
      #sim = sim + util.pytorch_cos_sim(desc_temas_res_embeddings[y], topics_embeddings[x])
      #sim = sim + topic_term[x].similarity(desc_temas_res[y])
    media.append(sim/10.0)
  return media

In [ ]:
#Similaridade Média usando Sentence Transformer v2 (distance.cosine do scipy.spatial)
 
def similaridade_Transformer_media_v2(topicos):
  from sentence_transformers import SentenceTransformer
  from scipy.spatial import distance
  model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

  desc_temas = description_full()
  #desc_temas_res = description_summarized()
  
  topics_embeddings = model.encode(topicos)
  desc_temas_embeddings = model.encode(desc_temas)
  #desc_temas_res_embeddings = model.encode(desc_temas_res)

  media = []
  for y in range(len(desc_temas_embeddings)):
    sim = 0.0
    for x in range(len(topics_embeddings)):
      sim = sim + (1 - distance.cosine(desc_temas_embeddings[y], topics_embeddings[x]))
      #sim = sim + topic_term[x].similarity(desc_temas_res[y])
    media.append(sim/10.0)
  return media

**Funções de similaridade - Utilizando o Maior valor de "similaridade" dos termos do Tópico em relação a descrição do Tema, comparar esses valores e classificar o acordão em relação ao maior valor de similaridade obtido em cada Tema.**   

In [ ]:
#Similaridade Maior usando Spacy

def similaridade_spacy_maior(topicos):  
  
  desc_temas = description_full()
  #desc_temas_res = description_summarized()
  maiores = []
  for y in range(len(desc_temas)):
  #for y in range(len(desc_temas_res)):
    maior = []
    for x in range(len(topicos)):
      tp = nlp(topicos[x])
      tm = nlp(desc_temas[y])
      #tm = nlp(desc_temas_res[y])
      maior.append(tp.similarity(tm))
    aux = max(maior)  
    maiores.append(aux)
  return maiores

In [ ]:
#Similaridade Maior usando Sentence Transformer v1 (pytorch)

def similaridade_Transformer_maior(topicos):
  from sentence_transformers import SentenceTransformer,util
  model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
  

  desc_temas = description_full()
  #desc_temas_res = description_summarized()
  
  topics_embeddings = model.encode(topicos)
  desc_temas_embeddings = model.encode(desc_temas)
  #desc_temas_res_embeddings = model.encode(desc_temas_res)processo_id

  maiores = []
  for y in range(len(desc_temas_embeddings)):
    maior = []
    for x in range(len(topics_embeddings)):
      maior.append(util.pytorch_cos_sim(desc_temas_embeddings[y], topics_embeddings[x]))
      #maior.append(topic_term[x].similarity(desc_temas_res[y]))
    aux = torch.max(torch.Tensor(maior))
    maiores.append(aux)
  return maiores

In [ ]:
#Similaridade Maior usando Sentence Transformer v2 (distance.cosine do scipy.spatial)

def similaridade_Transformer_maior_v2(topicos):
  from sentence_transformers import SentenceTransformer
  from scipy.spatial import distance
  model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

  desc_temas = description_full()
  #desc_temas_res = description_summarized()
  
  topics_embeddings = model.encode(topicos)
  desc_temas_embeddings = model.encode(desc_temas)
  #desc_temas_res_embeddings = model.encode(desc_temas_res)

  maiores = []
  for y in range(len(desc_temas_embeddings)):
    maior = []
    for x in range(len(topics_embeddings)):
      maior.append((1 - distance.cosine(desc_temas_embeddings[y], topics_embeddings[x])))
      #maior.append(topic_term[x].similarity(desc_temas_res[y]))
    aux = max(maior,int)
    maiores.append(aux)
  return maiores

In [ ]:
#Similaridade Mediana usando Spacy

def similaridade_spacy_mediana(top):  
  
  desc_temas = description_full()
  #desc_temas_res = description_summarized()
  mediana = []
  sim = []
  for y in range(len(desc_temas)):
  #for y in range(len(desc_temas_res)):
    #sim = 0.0
    for x in range(len(top)):
      tp = nlp(top[x])
      tm = nlp(desc_temas[y])
      #tm = nlp(desc_temas_res[y])
      sim.append(tp.similarity(tm))
    mediana.append(np.median(sim))
  return mediana

In [ ]:
#Similaridade Mediana usando Sentence Transformer v1 (pytorch)

def similaridade_Transformer_mediana(topicos):
  from sentence_transformers import SentenceTransformer,util
  model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
  

  desc_temas = description_full()
  #desc_temas_res = description_summarized()
  
  topics_embeddings = model.encode(topicos)
  desc_temas_embeddings = model.encode(desc_temas)
  #desc_temas_res_embeddings = model.encode(desc_temas_res)processo_id

  medianas = []
  for y in range(len(desc_temas_embeddings)):
    mediana = []
    for x in range(len(topics_embeddings)):
      mediana.append(util.pytorch_cos_sim(desc_temas_embeddings[y], topics_embeddings[x]))
      #maior.append(topic_term[x].similarity(desc_temas_res[y]))
    aux = torch.median(torch.Tensor(mediana))
    medianas.append(aux)
  return medianas

In [ ]:
#Similaridade Mediana usando Sentence Transformer v2 (distance.cosine do scipy.spatial)

def similaridade_Transformer_mediana_v2(topicos):
  from sentence_transformers import SentenceTransformer
  from scipy.spatial import distance
  model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

  desc_temas = description_full()
  #desc_temas_res = description_summarized()
  
  topics_embeddings = model.encode(topicos)
  desc_temas_embeddings = model.encode(desc_temas)
  #desc_temas_res_embeddings = model.encode(desc_temas_res)

  medianas = []
  for y in range(len(desc_temas_embeddings)):
    mediana = []
    for x in range(len(topics_embeddings)):
      mediana.append((1 - distance.cosine(desc_temas_embeddings[y], topics_embeddings[x])))
      #maior.append(topic_term[x].similarity(desc_temas_res[y]))
    aux = np.median(mediana)
    medianas.append(aux)
  return medianas

**Instalando o BERTopic**

In [ ]:
#!pip install bertopic

**MODELO DE TÓPICO (BERTopic)**

In [ ]:
def Modelo_Topico(text,var_id):
  umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
  hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)



  #model_path = 'ulysses-camara/legal-bert-pt-br'processo_id
  #sentence_model = SentenceTransformer(model_path)

  #topic_model = BERTopic(embedding_model=sentence_model, calculate_probabilities=True, verbose=True, n_gram_range=(2, 4))
  #topics, probs = topic_model.fit_transform(docs)

  topic_model = BERTopic(language="multilingual",  calculate_probabilities=True, verbose=True, n_gram_range=(4, 4), umap_model=umap_model, hdbscan_model=hdbscan_model) #n_gram_range=(4,4), calculate_probabilities=True)
  topics, probs = topic_model.fit_transform(text)


  #n_gram_range=(1, 1), min_topic_size=10, nr_topics=None, low_memory=False, calculate_probabilities=False, diversity=None, seed_topic_list=None, embedding_model=None, umap_model=None, hdbscan_model=None, vectorizer_model=None, verbose=False
  
  topicos = topic_model.get_topic(0) #Tópico mais frequente
  topic_term = []
  print(topicos)
  if(topicos != False):  #Caso do tópico de maior probabilidade de ocorrência, tópico 0 
    #return -1
      
    for i in range(len(topicos)):
      topic_term.append(nlp(topicos[i][0]))
    return topic_term
    
  else:   #Caso que ocorra só o outlier
    print(str(-1)+'\n')
    print(var_id)
    print('\n')
    
    topicos = topic_model.get_topic(-1)
    for i in range(len(topicos)):
      topic_term.append(nlp(topicos[i][0]))
    return topic_term
    
    #return var_id

In [ ]:
def Topic_Model_new(text,var_id):
    
    # Create TF-IDF sparse matrix
    vectorizer = TfidfVectorizer(min_df=5)
    embeddings = vectorizer.fit_transform(text)

    # Run BERTopic with embeddings
    topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True, n_gram_range=(4, 4))
    topics, probabilities = topic_model.fit_transform(text, embeddings)

    topicos = topic_model.get_topic(0) #Tópico mais frequente
    topic_term = []
    id_document_outlier = []
    print(topicos)
    if(topicos != False):  #Caso do tópico de maior probabilidade de ocorrência, tópico 0 
        #return -1
        
        for i in range(len(topicos)):
            topic_term.append(nlp(topicos[i][0]))
        return topic_term
        
    else:   #Caso que ocorra só o outlier
        id_document_outlier.append(var_id)
        print(str(-1)+'\n')
        print(var_id)
        print('\n')
        
        topicos = topic_model.get_topic(-1)
        for i in range(len(topicos)):
            topic_term.append(nlp(topicos[i][0]))
        return topic_term
    
  

In [ ]:
#df_clone é um dataframe com os acórdãos de entrada (coluna texto_pre_proc) 
#PRÉ-PROCESSAMENTO

#pp = Preprocessing(df = dff, name_column = 'formatado', threshold = 20, remove_stopwords = True, stopwords_language = "portuguese", ugc = False, stemming = False, remove_emojis = False, identation = True, html = False)
#df_new = pp.preprocess()
df_new = dff
df_new 


#SEGMENTAÇÃO
id_process = []

tops = []
acordao_pre = []
result_media = []
result_mediana = []
result_maior = []
for i in range(len(df_new)):
  print(i)    #Acórdãos Segmentados
  acordao_pre.append(Segmentation_v1(df_new['formatado'][i]))  # Coluna texto_pre_proc presente na base 2   
  #acordao_pre.append(Segmentation_v2(df_new['formatted'][i]))
  #tops.append(Modelo_Topico(acordao_pre[i],df_new['id_documento'][i]))                          #Extraindo os tópicos dos acordãos

  
  #aux = Modelo_Topico(acordao_pre[i],df_new['id_documento'][i])
  aux = Topic_Model_new(acordao_pre[i],df_new['id_documento'][i])
  tops.append(aux)
  

  #id_process.append(Modelo_Topico(acordao_pre[i],df_new['processo_id'][i]))

  #Média das Similaridades
  
  #media = similaridade_Transformer_media_v1(tops[i])                           #Calculando a Similaridade entre os termos do tópicos mais frequente com a Descrição do Tema
  #media = similaridade_Transformer_media_v1(tops[i])
  media = similaridade_spacy_media(tops[i])
  temas = [1101, 1039, 1033, 1015, 929]#, 1046]
  #print("ID do Acordão: ",df_new['id'][i])   # Coluna id presente na base 2
  #print("ID do Acordão: ",df_new['id_documento'][i])
  print("ID do Acordão: ",df_new.iloc[i].at['id_documento'])
  print("Média Similaridade Tema 1101: ",media[0])
  print("Média Similaridade Tema 1039: ",media[1])
  print("Média Similaridade Tema 1033: ",media[2])
  print("Média Similaridade Tema 1015: ",media[3])
  print("Média Similaridade Tema 929: ",media[4])
  #print("Média Similaridade Tema 1046: ",media[5])
  print("\nClassificação do acórdão junto ao Tema foi o",temas[media.index(max(media))])
  #result.append((df_new['id'][i], temas[media.index(max(media))])) # Coluna id presente na base 2
  #result.append((df_new['id_documento'][i], temas[media.index(max(media))])) # Coluna id presente na base 2 quando a necessidade do meu pré-processamento
  result_media.append((df_new.iloc[i].at['numero_processo'], df_new.iloc[i].at['id_documento'], temas[media.index(max(media))]))
  
  

  #Mediana das Similaridades


  #mediana = similaridade_spacy_mediana_v1(tops[i])                           #Calculando a Similaridade entre os termos do tópicos mais frequente com a Descrição do Tema
  #mediana = similaridade_Transformer_mediana(tops[i])
  mediana = similaridade_spacy_mediana(tops[i])
  temas = [1101, 1039, 1033, 1015, 929]#, 1046]
  #print("ID do Acordão: ",df_new['id'][i])   # Coluna id presente na base 2
  #print("ID do Acordão: ",df_new['id_documento'][i])
  print("ID do Acordão: ",df_new.iloc[i].at['id_documento'])
  print("Mediana Similaridade Tema 1101: ",mediana[0])
  print("Mediana Similaridade Tema 1039: ",mediana[1])
  print("Mediana Similaridade Tema 1033: ",mediana[2])
  print("Mediana Similaridade Tema 1015: ",mediana[3])
  print("Mediana Similaridade Tema 929: ",mediana[4])
  #print("Mediana Similaridade Tema 1046: ",mediana[5])
  print("\nClassificação do acórdão junto ao Tema foi o",temas[mediana.index(max(mediana))])
  #result.append((df_new['id'][i], temas[media.index(max(media))])) # Coluna id presente na base 2
  #result.append((df_new['id_documento'][i], temas[media.index(max(media))])) # Coluna id presente na base 2 quando a necessidade do meu pré-processamento
  result_mediana.append((df_new.iloc[i].at['numero_processo'], df_new.iloc[i].at['id_documento'], temas[mediana.index(max(mediana))]))

  
  #Maior valor de Similaridade
  
  #maiores = similaridade_Transformer_maior(tops[i])
  maiores = similaridade_spacy_maior(tops[i])
  temas = [1101, 1039, 1033, 1015, 929]#, 1046]
  #print("ID do Acordão: ",df_new['id'][i])   # Coluna id presente na base 2
  #print("ID do Acordão: ",df_new['processo_id'][i])
  print("ID do Acordão: ",df_new.iloc[i].at['id_documento'])
  print("Maior Valor Similaridade Tema 1101: ",maiores[0])
  print("Maior Valor Similaridade Tema 1039: ",maiores[1])
  print("Maior Valor Similaridade Tema 1033: ",maiores[2])
  print("Maior Valor Similaridade Tema 1015: ",maiores[3])
  print("Maior Valor Similaridade Tema 929: ",maiores[4])
  #print("Maior Valor Similaridade Tema 1046: ",maiores[5])
  print("\nClassificação do acórdão junto ao Tema foi o",temas[maiores.index(max(maiores))])
  #result.append((df_new['id'][i], temas[maiores.index(max(maiores))]))   # Coluna id presente na base 2
  result_maior.append((df_new.iloc[i].at['numero_processo'], df_new.iloc[i].at['id_documento'], temas[maiores.index(max(maiores))]))
  
Classification_result(df_new, result_media, 'Media_spacy_seg1_comStopW.css') #Experimento: maior Media das similaridades, Segmento 2 e com Stop Word
Classification_result(df_new, result_maior, 'Maior_spacy_seg1_comStopW.css') #Experimento: maior das Maiores similaridades, Segmento 2 e com Stop Word
Classification_result(df_new, result_mediana, 'Mediana_spacy_seg1_comStopW.css') #Experimento: maior Mediana das similaridades, Segmento 2 e com Stop Word  

0


2023-03-22 15:07:35,519 - BERTopic - Reduced dimensionality
2023-03-22 15:07:35,529 - BERTopic - Clustered reduced embeddings


[('cível nº 0000150 84', 0.01666972218726992), ('0000150 84 2015 26', 0.01666972218726992), ('84 2015 26 0315', 0.01666972218726992), ('nº 0000150 84 2015', 0.01666972218726992), ('apelação cível nº 0000150', 0.01666972218726992), ('26 0315 laranjal paulista', 0.0157916559646318), ('0315 laranjal paulista 15', 0.0157916559646318), ('2015 26 0315 laranjal', 0.0157916559646318), ('ação civil pública execução', 0.013049256057137424), ('civil pública execução individual', 0.01209376559494789)]
ID do Acordão:  76846788 - 0
Média Similaridade Tema 1101:  0.15620302549068127
Média Similaridade Tema 1039:  0.13943740594217316
Média Similaridade Tema 1033:  0.1625053048522263
Média Similaridade Tema 1015:  0.15461270672924946
Média Similaridade Tema 929:  0.2326739135888071

Classificação do acórdão junto ao Tema foi o 929
ID do Acordão:  76846788 - 0
Mediana Similaridade Tema 1101:  0.04825948099186335
Mediana Similaridade Tema 1039:  0.03212377287491156
Mediana Similaridade Tema 1033:  0.0346

2023-03-22 15:07:38,524 - BERTopic - Reduced dimensionality
2023-03-22 15:07:38,530 - BERTopic - Clustered reduced embeddings


[('os chamados vícios de', 0.010721271962238509), ('chamados vícios de construção', 0.010721271962238509), ('do sistema financeiro de', 0.010721271962238509), ('sistema financeiro de habitação', 0.010721271962238509), ('45 2015 26 0691', 0.00770381745876783), ('estado de são paulo', 0.00770381745876783), ('nº 0000349 45 2015', 0.00770381745876783), ('companhia excelsior de seguros', 0.00770381745876783), ('do seguro compreensivo especial', 0.00770381745876783), ('determinação de regular prosseguimento', 0.00770381745876783)]
ID do Acordão:  55161519 - 0
Média Similaridade Tema 1101:  0.5813556653909773
Média Similaridade Tema 1039:  0.5886108075307614
Média Similaridade Tema 1033:  0.5878992334247658
Média Similaridade Tema 1015:  0.5557071468845786
Média Similaridade Tema 929:  0.5199781622072032

Classificação do acórdão junto ao Tema foi o 1039
ID do Acordão:  55161519 - 0
Mediana Similaridade Tema 1101:  0.6757509801153266
Mediana Similaridade Tema 1039:  0.70612268374862
Mediana S

...

[('código de processo civil', 0.008919138016843225), ('inflacionários ação civil pública', 0.008919138016843225), ('expurgos inflacionários ação civil', 0.008919138016843225), ('ação civil pública execução', 0.008919138016843225), ('do código de processo', 0.008919138016843225), ('da ação civil pública', 0.0083366027247592), ('civil pública execução individual', 0.0083366027247592), ('instrumento expurgos inflacionários ação', 0.007738949015155333), ('de instrumento expurgos inflacionários', 0.007578729749781091), ('agravo de instrumento expurgos', 0.007578729749781091)]
ID do Acordão:  111462230 - 0
Média Similaridade Tema 1101:  0.6867447490647087
Média Similaridade Tema 1039:  0.6971348724232809
Média Similaridade Tema 1033:  0.7249743891683558
Média Similaridade Tema 1015:  0.6748466785729177
Média Similaridade Tema 929:  0.598430852159941

Classificação do acórdão junto ao Tema foi o 1033
ID do Acordão:  111462230 - 0
Mediana Similaridade Tema 1101:  0.655060528380853
Mediana Simi

2023-03-22 16:26:23,963 - BERTopic - Reduced dimensionality
2023-03-22 16:26:23,969 - BERTopic - Clustered reduced embeddings


[('câmara de direito privado', 0.023449124641768397), ('de direito privado rel', 0.020406817252566795), ('mediante agravo de instrumento', 0.017175738903253725), ('4008541 13 2013 26', 0.017175738903253725), ('13 2013 26 0320', 0.017175738903253725), ('decisão que resolver impugnação', 0.017175738903253725), ('agravo de instrumento salvo', 0.017175738903253725), ('resolver impugnação recorrível mediante', 0.01370638085506392), ('extinção da execução caso', 0.01370638085506392), ('execução caso em que', 0.01370638085506392)]
ID do Acordão:  25773188 - 0
Média Similaridade Tema 1101:  0.5203958002156343
Média Similaridade Tema 1039:  0.5257719278121092
Média Similaridade Tema 1033:  0.5626656297092525
Média Similaridade Tema 1015:  0.5132096806230094
Média Similaridade Tema 929:  0.4942856746760665

Classificação do acórdão junto ao Tema foi o 1033
ID do Acordão:  25773188 - 0
Mediana Similaridade Tema 1101:  0.6296224671658359
Mediana Similaridade Tema 1039:  0.6379856717758232
Mediana 

2023-03-22 16:26:26,946 - BERTopic - Reduced dimensionality
2023-03-22 16:26:26,953 - BERTopic - Clustered reduced embeddings


[('câmara de direito privado', 0.01730525754520116), ('de direito privado do', 0.01730525754520116), ('privado do tjsp julgado', 0.01730525754520116), ('do tjsp julgado em', 0.01730525754520116), ('direito privado do tjsp', 0.01730525754520116), ('fundo de compensação de', 0.009835529014109202), ('de compensação de variações', 0.009835529014109202), ('compensação de variações salariais', 0.009835529014109202), ('sistema financeiro da habitação', 0.009835529014109202), ('10ª câmara de direito', 0.009835529014109202)]
ID do Acordão:  14629562 - 0
Média Similaridade Tema 1101:  0.6712941855486225
Média Similaridade Tema 1039:  0.7607698194192384
Média Similaridade Tema 1033:  0.7593941080009194
Média Similaridade Tema 1015:  0.6760353468202247
Média Similaridade Tema 929:  0.6723653701710488

Classificação do acórdão junto ao Tema foi o 1039
ID do Acordão:  14629562 - 0
Mediana Similaridade Tema 1101:  0.7096938297099777
Mediana Similaridade Tema 1039:  0.7382160719963985
Mediana Similari

In [ ]:
result_maior

[('00001508420158260315', '76846788 - 0', 1033),
 ('00003494520158260691', '55161519 - 0', 1039),
 ('00004868820058260590', '12628095 - 0', 1033),
 ('00005505720218260196', '114316205 - 0', 1033),
 ('00005702020168260262', '55185431 - 0', 1039),
 ('00005710520168260262', '45143383 - 0', 1039),
 ('00006081820148260160', '116259837 - 0', 1101),
 ('00009615420148260420', '111442397 - 0', 1033),
 ('00009943920128260024', '77087363 - 0', 1033),
 ('00011016320148260493', '108704093 - 0', 1039),
 ('00012105820148260370', '119014975 - 0', 1101),
 ('00013534320158260457', '88617733 - 0', 1101),
 ('00015744420148260042', '108372372 - 0', 1033),
 ('00016420820118260136', '5361714 - 0', 1033),
 ('00017940320108260165', '107507815 - 0', 1039),
 ('00018005120118260431', '77916928 - 0', 1033),
 ('00019006320118260315', '123535659 - 0', 1039),
 ('00021848320148260083', '110417372 - 0', 1039),
 ('00022353720158260123', '105873464 - 0', 1039),
 ('00022901720118260288', '59216619 - 0', 1033),
 ('00023794